In [10]:
!pip3 install tensorflow

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/85/15/cf99a373812d37f8ae99752a34a9f5f690d820ceb5b302e922705bc18944/tensorflow-2.15.0-cp311-cp311-macosx_12_0_arm64.whl.metadata
  Obtaining dependency information for tensorflow-macos==2.15.0 from https://files.pythonhosted.org/packages/eb/9f/0759e2fea4a3c48f070b64811c2c57036b46353ba87263afc810b8f4188a/tensorflow_macos-2.15.0-cp311-cp311-macosx_12_0_arm64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/01/e4/dc0a1dcc4e74e08d7abedab278c795eef54a224363bb18f5692f416d834f/absl_py-2.0.0-py3-none-any.whl.metadata
  Using cached absl_py-2.0.0-py3-none-any.whl.metadata (2.3 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 1.7 MB/s eta 0:00:00
  Obtaining dependency information for libclang>=13.0.0 from https://files.pythonhosted.org/packages/32/1f/981809b77b71972beec34b3ff5422c1b1f7e519daac7b3cbd055c05ba2cf/libclang-16.0.6-

In [55]:
# pip install tensorflow==2.9.1
# pip install imblearn

# from src.ml_modules.deep_learning import DeepLearning
# from src.repository.dataframe_loader import load_dataframe
# from src.ml_modules.model_testing import train_and_test_selected_model
# from src.constants import STANDARD_FEATURES
# from src.helper.databricks_configuration import spark
# from src.helper.model_saver import save_model, load_model

import pandas as pd
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import category_encoders as ce


In [62]:
filename = "flight_data_weather.csv"
filepath = "../data/"
flight_df = pd.read_csv(filepath + filename)

In [63]:
flight_df.columns

Index(['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 'FlightDate',
       'Reporting_Airline', 'Tail_Number', 'Flight_Number_Reporting_Airline',
       'Origin', 'Dest', 'DepTime', 'DepDelay', 'TaxiOut', 'WheelsOff',
       'WheelsOn', 'TaxiIn', 'CRSArrTime', 'ArrTime', 'ArrDelay', 'ArrDel15',
       'CRSElapsedTime', 'ActualElapsedTime', 'AirTime', 'Distance', 'Carrier',
       'Full-time', 'Part-time', 'Grand Total', 'Origin_Windspeed',
       'Origin_Precip', 'Dest_Windspeed', 'Dest_Precip', 'dest_ianaTimeZone',
       'origin_ianaTimeZone', 'Aircraft_Daily_Flight_Count'],
      dtype='object')

In [64]:
flight_df.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,Origin,...,Full-time,Part-time,Grand Total,Origin_Windspeed,Origin_Precip,Dest_Windspeed,Dest_Precip,dest_ianaTimeZone,origin_ianaTimeZone,Aircraft_Daily_Flight_Count
0,2020.0,1.0,1.0,1.0,3.0,2020-01-01,AA,N407AN,664.0,KOA,...,95612.0,11840.0,107452.0,3.0,0.08,0.0,0.0,America/Phoenix,America/Los_Angeles,1
1,2020.0,1.0,1.0,1.0,3.0,2020-01-01,9E,N297PQ,5270.0,LFT,...,4741.0,103.0,4844.0,0.0,0.00,5.0,0.0,America/New_York,America/Chicago,1
2,2020.0,1.0,1.0,1.0,3.0,2020-01-01,WN,N963WN,1395.0,CMH,...,59914.0,2302.0,62216.0,12.0,0.00,7.0,0.0,America/Chicago,America/New_York,1
3,2020.0,1.0,1.0,1.0,3.0,2020-01-01,AA,N891NN,137.0,TUS,...,95612.0,11840.0,107452.0,7.0,0.00,8.0,0.0,America/Chicago,America/Phoenix,1
4,2020.0,1.0,1.0,1.0,3.0,2020-01-01,AA,N807AW,2189.0,SAT,...,95612.0,11840.0,107452.0,0.0,0.00,12.0,0.0,America/Chicago,America/Chicago,1


In [65]:
flight_df.describe()

,Year,Quarter,Month,DayofMonth,DayOfWeek,Flight_Number_Reporting_Airline,DepTime,DepDelay,TaxiOut,WheelsOff,...,AirTime,Distance,Full-time,Part-time,Grand Total,Origin_Windspeed,Origin_Precip,Dest_Windspeed,Dest_Precip,Aircraft_Daily_Flight_Count
count,343120.000000,343120.000000,343120.000000,343120.000000,343120.000000,343120.000000,343120.000000,343120.000000,343120.000000,343120.000000,...,343120.000000,343120.000000,343120.000000,343120.000000,343120.000000,343120.000000,343120.000000,343120.000000,343120.000000,343120.000000
mean,2021.003427,2.502361,6.506942,15.745570,4.034481,2582.760463,1386.532359,28.711952,17.955564,1410.125017,...,112.987748,804.425530,41425.298476,3935.257814,45360.556289,8.809997,0.003896,8.753518,0.001704,1.049910
std,0.816050,1.118062,3.452625,8.741097,2.008426,1774.133707,492.737973,74.270357,11.886630,495.218111,...,67.377404,558.944366,32796.343874,4397.024946,36010.527201,5.556166,0.033748,5.541260,0.067881,0.228342
min,2020.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-62.000000,1.000000,1.000000,...,11.000000,29.000000,2357.000000,0.000000,2374.000000,0.000000,0.000000,-3.000000,-3.000000,1.000000
25%,2020.000000,2.000000,4.000000,8.000000,2.000000,1081.000000,1003.000000,-4.000000,11.000000,1020.000000,...,63.000000,395.000000,11030.000000,740.000000,13191.000000,5.000000,0.000000,5.000000,0.000000,1.000000
50%,2021.000000,3.000000,7.000000,16.000000,4.000000,2211.000000,1412.000000,2.000000,14.000000,1426.000000,...,98.000000,674.000000,53269.000000,1821.000000,54748.000000,8.000000,0.000000,8.000000,0.000000,1.000000
75%,2022.000000,4.000000,10.000000,23.000000,6.000000,3944.000000,1806.000000,37.000000,20.000000,1822.000000,...,143.000000,1040.000000,65881.000000,5655.000000,72639.000000,12.000000,0.000000,12.000000,0.000000,1.000000
max,2022.000000,4.000000,12.000000,31.000000,7.000000,8819.000000,2400.000000,2175.000000,218.000000,2400.000000,...,677.000000,5095.000000,97373.000000,16424.000000,109108.000000,86.000000,2.390000,86.000000,3.870000,5.000000


In [23]:
# flight_df['Aircraft_Daily_Flight_Count'] = flight_df['Aircraft_Daily_Flight_Count'].astype('float64')

In [66]:
features = ['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 
       'DepTime', 'DepDelay','ArrTime',
       'Distance', 'Carrier', 'Origin_Windspeed', 'Origin_Precip', 
       'Dest_Windspeed', 'Dest_Precip', 'dest_ianaTimeZone',
       'origin_ianaTimeZone', 'Aircraft_Daily_Flight_Count']
len(features)

17

In [68]:
# Reducing dimensionality of Carrier
value_counts = flight_df['Carrier'].value_counts()
to_remove = value_counts[value_counts <= 9000].index
flight_df['Carrier'].replace(to_remove, 'Other', inplace=True)

In [60]:
flight_df.columns

Index(['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 'FlightDate',
       'Reporting_Airline', 'Tail_Number', 'Flight_Number_Reporting_Airline',
       'Origin', 'Dest', 'DepTime', 'DepDelay', 'TaxiOut', 'WheelsOff',
       'WheelsOn', 'TaxiIn', 'CRSArrTime', 'ArrTime', 'ArrDelay', 'ArrDel15',
       'CRSElapsedTime', 'ActualElapsedTime', 'AirTime', 'Distance', 'Carrier',
       'Full-time', 'Part-time', 'Grand Total', 'Origin_Windspeed',
       'Origin_Precip', 'Dest_Windspeed', 'Dest_Precip',
       'Aircraft_Daily_Flight_Count', 'origin_ianaTimeZone_0',
       'origin_ianaTimeZone_1', 'origin_ianaTimeZone_2',
       'origin_ianaTimeZone_3', 'dest_ianaTimeZone_0', 'dest_ianaTimeZone_1',
       'dest_ianaTimeZone_2', 'dest_ianaTimeZone_3'],
      dtype='object')

In [69]:
y = flight_df['ArrDel15']
X = flight_df[features]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [77]:
# Reducing dimensionality of TimeZones 
def binary_encode_timezone(df):
    encoder = ce.BinaryEncoder(cols=['origin_ianaTimeZone', 'dest_ianaTimeZone'])

    # Fit and transform to produce binary encoded data
    df_encoded = encoder.fit_transform(df[['origin_ianaTimeZone', 'dest_ianaTimeZone']])

    # Merge the encoded data back with the original DataFrame
    df = df.drop(['origin_ianaTimeZone', 'dest_ianaTimeZone'], axis=1)
    df = pd.concat([df, df_encoded], axis=1)
    return df

In [78]:
X_train = binary_encode_timezone(X_train)
X_test = binary_encode_timezone(X_test)

In [79]:
X_train = pd.get_dummies(X_train, columns=['Carrier'])
X_test = pd.get_dummies(X_test, columns=['Carrier'])
X_train.shape

In [72]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # or 'softmax' for multi-class classification
])

In [73]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [75]:
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
6863/6863 [==============================] - 3s 468us/step - loss: 1.1617 - accuracy: 0.8092 - val_loss: 0.3531 - val_accuracy: 0.8643
Epoch 2/10
6863/6863 [==============================] - 3s 440us/step - loss: 0.4288 - accuracy: 0.8370 - val_loss: 0.3397 - val_accuracy: 0.8513
Epoch 3/10
6863/6863 [==============================] - 3s 433us/step - loss: 0.3449 - accuracy: 0.8543 - val_loss: 0.3346 - val_accuracy: 0.8488
Epoch 4/10
6863/6863 [==============================] - 3s 431us/step - loss: 0.3190 - accuracy: 0.8632 - val_loss: 0.3110 - val_accuracy: 0.8685
Epoch 5/10
6863/6863 [==============================] - 3s 435us/step - loss: 0.3078 - accuracy: 0.8667 - val_loss: 0.2928 - val_accuracy: 0.8734
Epoch 6/10
6863/6863 [==============================] - 3s 452us/step - loss: 0.3035 - accuracy: 0.8686 - val_loss: 0.2957 - val_accuracy: 0.8734
Epoch 7/10
6863/6863 [==============================] - 3s 439us/step - loss: 0.3016 - accuracy: 0.8692 - val_loss: 0.2903 -

In [81]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)
print('Test accuracy:', test_loss)

2145/2145 [==============================] - 1s 253us/step - loss: 0.3142 - accuracy: 0.8607
Test accuracy: 0.8607338666915894
Test accuracy: 0.3141610026359558
